In [11]:
import numpy as np
import pandas as pd
from scrapy.selector import Selector
from selenium import webdriver 
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [92]:
driver = webdriver.Chrome()
url = "https://talk.washingtonpost.com/embed/stream?storyURL=https%3A%2F%2Fwww.washingtonpost.com%2Fopinions%2F2023%2F01%2F04%2Fheat-pumps-climate-carbon-emission-revolution%2F&v=6.16.2&ts=1699910100000&initialWidth=640&childId=comments&parentTitle=Opinion%20%7C%20Heat%20pumps%20could%20be%20a%20climate%20policy%20revolution%20—%20if%20we%20let%20them%20-%20The%20Washington%20Post&parentUrl=https%3A%2F%2Fwww.washingtonpost.com%2Fopinions%2F2023%2F01%2F04%2Fheat-pumps-climate-carbon-emission-revolution%2F"
driver.get(url)
time.sleep(5)
while True:
    try:
        # Identify and click the "Load More" button
        load_more_button = driver.find_element(By.ID, 'comments-loadMore')
        time.sleep(2)
        load_more_button.click()
        time.sleep(5)
    
    except Exception as e:
        # If the "Load More" button is not found, break out of the loop
        break

elements = driver.find_elements(By.CSS_SELECTOR, '[id^="comment"]')
comments_text = elements[0].text

comments=comments_text.split("share\n")

for i in range(len(comments)-1):
    comments[i] = comments[i].split("ago\n")[1]

comments = [c for c in comments if c!="READ MORE OF THIS CONVERSATION >"]
comments = [c.replace("\n", "") for c in comments]

for i in range(len(comments)-1):
    if comments[i][0:8]=="(Edited)":
        comments[i] = comments[i][8:]

print("Before eliminating short comments, we have a total of:", len(comments), "comments.")
        
comments_long = [c for c in comments if len(c)>=20]

print("After eliminating short comments, we have a total of:", len(comments_long), "comments.")

driver.quit()

Before eliminating short comments, we have a total of: 3040 comments.
After eliminating short comments, we have a total of: 2956 comments.
['Well, to reduce strains on the grid both install heat pumps (should be required on all new buildings nationally), and solar panels on buildings. To attempt to clean up the global environment though reversing the global population explosion an obvious factor that is never talked about. Have to wonder how much "covert" censorship is going on in corporate owned mass media that the global population issue (from 2.5 to 8 billion since 1950) is never discussed. In fact you will see WaPo columnists try to scare us, because a few developed countries are having less children, that it is bad economically to have fewer children.', 'I’d like to replace my gas burning boiler but not the radiator system. Is there any hope for an electric boiler?', 'Better chance for a hydrogen powered one. The first domestic version was installed in the Netherlands in 2019:http

In [98]:
import csv

with open('comments_long.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Comments'])
    for item in comments_long:
        writer.writerow([item])

with open('comments.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Comments'])
    for item in comments:
        writer.writerow([item])

In [100]:
import pandas as pd
pd.read_csv("comments_long.csv")

,Comments
0,"Well, to reduce strains on the grid both insta..."
1,I’d like to replace my gas burning boiler but ...
2,Better chance for a hydrogen powered one. The ...
3,Stressing the electric grid more before shorin...
4,End fossil fuel use NOW!
...,...
2951,Battery backup for a PV system is very expensi...
2952,That’s a perfectly reasonable route. I don’t h...
2953,Think wood burning stove. Or a back up generator.
2954,You mean like a fireplace or a Franklin stove?...
